In [1]:
from decouple import config

In [2]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [3]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [4]:
import requests

api_key = ALPHA_VANTAGE_API_KEY

params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY"
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=1min&apikey={api_key}'.format(
    **params
)
r = requests.get(url)
data = r.json()

In [5]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (1min)'

In [6]:
results = data[dataset_key] # dict
# for timestamp_str in results.keys():
#     print(timestamp_str, results.get(timestamp_str))

In [7]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2024-11-22 19:59:00'

In [8]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '229.9350',
 '2. high': '229.9500',
 '3. low': '229.9200',
 '4. close': '229.9350',
 '5. volume': '169'}

In [9]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result):
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S' 
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str,timestamp_format)).astimezone(utc)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }

In [10]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('229.9350'),
 'close_price': Decimal('229.9350'),
 'high_price': Decimal('229.9500'),
 'low_price': Decimal('229.9200'),
 'number_of_trades': None,
 'volume': 169,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 11, 23, 0, 59, tzinfo=<UTC>)}

In [ ]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))

In [12]:
new_data

{'open_price': Decimal('229.9100'),
 'close_price': Decimal('229.9450'),
 'high_price': Decimal('229.9450'),
 'low_price': Decimal('229.9100'),
 'number_of_trades': None,
 'volume': 4,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 11, 22, 23, 20, tzinfo=<UTC>)}